## **Groq**

Groq is a natural language processing framework that enables structured interaction between large language models (LLMs) and external tools or APIs. It provides a standardized interface to define and manage tool calls—functions or services invoked by the LLM to handle specialized tasks.

### **Introduction to Tool Use**

Tool use is a powerful feature that allows Large Language Models (LLMs) to interact with external resources, such as APIs, databases, and the web, to gather dynamic data they wouldn't otherwise have access to in their pre-trained (or static) state and perform actions beyond simple text generation.

Tool use bridges the gap between the data that the LLMs were trained on with dynamic data and real-world actions, which opens up a wide array of realtime use cases for us to build applications with.

#### **How Tool Use Works:**

To integrate tools with Groq API, follow these steps:

1. Provide tools (or predefined functions) to the LLM for performing actions and accessing external data in real-time in addition to your user prompt within your Groq API request.
2. Define how the tools should be used to teach the LLM how to use them effectively (e.g. by defining input and output formats).
3. Let the LLM autonomously decide whether or not the provided tools are needed for a user query by evaluating the user query, determining whether the tools can enhance its response, and utilizing the tools accordingly.
4. Extract tool input, execute the tool code, and return results.
5. Let the LLM use the tool result to formulate a response to the original prompt.

This process allows the LLM to perform tasks such as real-time data retrieval, complex calculations, and external API interaction, all while maintaining a natural conversation with our end user.

#### **Tool Use with Groq**

Groq API endpoints support tool use to almost instantly deliver structured JSON output that can be used to directly invoke functions from desired external resources.

- **Supported Models:** 

Llama 3.3 and 3.1 Models

The following Llama-3.3 models are recommended for tool use due to their versatility and performance:

- llama-3.3-70b-versatile
- llama-3.1-8b-instant


- **Other Supported Models:**

The following models powered by Groq also support tool use:

- llama3-70b-8192
- llama3-8b-8192
- mixtral-8x7b-32768 (parallel tool use not supported)
- gemma2-9b-it (parallel tool use not supported)

#### **Tools Specifications**

Tool Call and Tool Response Structure: 

**Tool Call Structure:**

```
{
  "model": "llama-3.3-70b-versatile",
  "messages": [
    {
      "role": "system",
      "content": "You are a weather assistant. Use the get_weather function to retrieve weather information for a given location."
    },
    {
      "role": "user",
      "content": "What's the weather like in New York today?"
    }
  ],
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "get_weather",
        "description": "Get the current weather for a location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g. San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": ["celsius", "fahrenheit"],
              "description": "The unit of temperature to use. Defaults to fahrenheit."
            }
          },
          "required": ["location"]
        }
      }
    }
  ],
  "tool_choice": "auto",
  "max_tokens": 4096
}'
```

**Tool Call Response:**

```
"model": "llama-3.3-70b-versatile",
"choices": [{
    "index": 0,
    "message": {
        "role": "assistant",
        "tool_calls": [{
            "id": "call_d5wg",
            "type": "function",
            "function": {
                "name": "get_weather",
                "arguments": "{\"location\": \"New York, NY\"}"
            }
        }]
    },
    "logprobs": null,
    "finish_reason": "tool_calls"
}],
```

When a model decides to use a tool, it returns a response with a tool_calls object containing:

- id: a unique identifier for the tool call
- type: the type of tool call, i.e. function
- name: the name of the tool being used
- parameters: an object containing the input being passed to the tool

## **Setting Up Tools with Groq**

Note: In this example, we're defining a function as our tool, but your tool can be any function or an external resource (e.g. dabatase, web search engine, external API).

In [ ]:
# Groq Tool Calling
import os
import json
from groq import Groq

from dotenv import load_dotenv

# Initialize the Groq client
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
client = Groq(api_key=groq_api_key)

# Specify the model to be used 
MODEL = 'llama-3.3-70b-versatile'

# Step 1: Create tool
def calculate(expression):
    """
    Evaluate a mathematical expression
    """
    try:
        # Attempt to evaluate the math expression
        result = eval(expression)
        return json.dumps({"result": result})

    except:
        # Return an error message if the math expression is invalid
        return json.dumps({"error": "Invalid expression"})

# Step 2: Pass Tool Definition and Messages to Model
# Step 3: Receive and Handle Tool Results    
def run_conversation(user_prompt):
    # Initialize the conversation with system and user messages
    messages=[
        {
            "role": "system",
            "content": "You are a calculator assistant. Use the calculate function to perform mathematical operations and provide the results."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    # Define the available tools (i.e. functions) for our model to use
    tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate",
                "description": "Evaluate a mathematical expression",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "The mathematical expression to evaluate",
                        }
                    },
                    "required": ["expression"],
                },
            },
        }
    ]
    # Make the initial API call to Groq
    response = client.chat.completions.create(
        model=MODEL, # LLM to use
        messages=messages, # Conversation history
        stream=False,
        tools=tools, # Available tools (i.e. functions) for our LLM to use
        tool_choice="auto", # Let our LLM decide when to use tools
        max_tokens=4096 # Maximum number of tokens to allow in our response
    )
    # Extract the response and any tool call responses
    response_message = response.choices[0].message
    print('response_message', response_message)
    tool_calls = response_message.tool_calls
    if tool_calls:
        # Define the available tools that can be called by the LLM
        available_functions = {
            "calculate": calculate,
        }
        # Add the LLM's response to the conversation
        messages.append(response_message)

        # Process each tool call
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            # Call the tool and get the response
            function_response = function_to_call(
                expression=function_args.get("expression")
            )
            # Add the tool response to the conversation
            messages.append(
                {
                    "tool_call_id": tool_call.id, 
                    "role": "tool", # Indicates this message is from tool use
                    "name": function_name,
                    "content": function_response,
                }
            )
        # Make a second API call with the updated conversation
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        # Return the final response
        return second_response.choices[0].message.content
# Example usage
user_prompt = "What is 25 * 4 + 10?"
print(run_conversation(user_prompt))

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


response_message ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_m8z6', function=Function(arguments='{"expression": "25 * 4 + 10"}', name='calculate'), type='function')])


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


The result of 25 * 4 + 10 is 110.


## **Routing System**

It's possible to use different available models as part of a routing system:

1. **Query Analysis**: Implement a routing system that analyzes incoming user queries to determine their nature and requirements.
2. **Model Selection**: Based on the query analysis, route the request to the most appropriate model:
    - For queries involving function calling, API interactions, or structured data manipulation, use the Llama 3 Groq Tool Use models.
    - For general knowledge, open-ended conversations, or tasks not specifically related to tool use, route to a general-purpose language model, such as Llama 3 70B.

In [ ]:
# Define models
ROUTING_MODEL = "llama3-70b-8192"
TOOL_USE_MODEL = "llama-3.3-70b-versatile"
GENERAL_MODEL = "llama3-70b-8192"

def calculate(expression):
    """
    Tool to evaluate a mathematical expression
    """
    try:
        result = eval(expression)
        return json.dumps({"result": result})
    except:
        return json.dumps({"error": "Invalid expression"})

def route_query(query):
    """Routing logic to let LLM decide if tools are needed"""
    routing_prompt = f"""
    Given the following user query, determine if any tools are needed to answer it.
    If a calculation tool is needed, respond with 'TOOL: CALCULATE'.
    If no tools are needed, respond with 'NO TOOL'.

    User query: {query}

    Response:
    """
    
    response = client.chat.completions.create(
        model=ROUTING_MODEL,
        messages=[
            {"role": "system", "content": "You are a routing assistant. Determine if tools are needed based on the user query."},
            {"role": "user", "content": routing_prompt}
        ],
        max_tokens=20  # We only need a short response
    )
    
    routing_decision = response.choices[0].message.content.strip()
    
    if "TOOL: CALCULATE" in routing_decision:
        return "calculate tool needed"
    else:
        return "no tool needed"

def run_with_tool(query):
    """
    Use the tool use model to perform the calculation
    """
    messages = [
        {
            "role": "system",
            "content": "You are a calculator assistant. Use the calculate function to perform mathematical operations and provide the results.",
        },
        {
            "role": "user",
            "content": query,
        }
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate",
                "description": "Evaluate a mathematical expression",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "The mathematical expression to evaluate",
                        }
                    },
                    "required": ["expression"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model=TOOL_USE_MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    if tool_calls:
        messages.append(response_message)
        for tool_call in tool_calls:
            function_args = json.loads(tool_call.function.arguments)
            function_response = calculate(function_args.get("expression"))
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": "calculate",
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(
            model=TOOL_USE_MODEL,
            messages=messages
        )
        return second_response.choices[0].message.content
    return response_message.content

def run_general(query):
    """
    Use the general model to answer the query since no tool is needed
    """
    response = client.chat.completions.create(
        model=GENERAL_MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": query}
        ]
    )
    return response.choices[0].message.content

def process_query(query):
    """
    Process the query and route it to the appropriate model
    """
    route = route_query(query)
    if route == "calculate":
        response = run_with_tool(query)
    else:
        response = run_general(query)
    
    return {
        "query": query,
        "route": route,
        "response": response
    }

# Example usage
queries = [
    "What is the capital of the Netherlands?",
    "Calculate 25 * 4 + 10"
]

for query in queries:
    result = process_query(query)
    print(f"Query: {result['query']}")
    print(f"Route: {result['route']}")
    print(f"Response: {result['response']}\n")

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Query: What is the capital of the Netherlands?
Route: no tool needed
Response: The capital of the Netherlands is Amsterdam!



INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Query: Calculate 25 * 4 + 10
Route: calculate tool needed
Response: To calculate this, I'll follow the order of operations (PEMDAS):

1. Multiply 25 and 4: 25 * 4 = 100
2. Add 10 to the result: 100 + 10 = 110

So the final answer is 110!



### **DuckDuckGo**

DuckDuckGo is a privacy-focused search engine designed to provide search results without tracking users' personal data. It emphasizes anonymity and delivers results through a combination of its own indexing and external sources like Bing and Wikipedia.

- The DuckDuckGo Search (DDGS) API allows developers to fetch search results programmatically.
- Supports text search, instant answers, and up to five results per query.

In [ ]:
# Using duckduckgo 
from duckduckgo_search import DDGS

results = DDGS().text('Curitiba temperature', max_results=5)
print(results)

INFO:primp:response: https://lite.duckduckgo.com/lite/ 200 15746


[{'title': 'Curitiba, Paraná, Brazil Weather Forecast | AccuWeather', 'href': 'https://www.accuweather.com/en/br/curitiba/44944/weather-forecast/44944', 'body': 'Curitiba, Paraná, Brazil Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days.'}, {'title': 'Weather in Curitiba, Paraná, Brazil - timeanddate.com', 'href': 'https://www.timeanddate.com/weather/brazil/curitiba', 'body': 'Current weather in Curitiba and forecast for today, tomorrow, and next 14 days'}, {'title': 'Curitiba, Brazil Weather Conditions | Weather Underground', 'href': 'https://www.wunderground.com/weather/br/curitiba', 'body': 'Curitiba Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the Curitiba area.'}, {'title': 'Curitiba, Paraná, Brazil Current Weather | AccuWeather', 'href': 'https://www.accuweather.com/en/br/curitiba/44944/current-weather/44944', 'body': 'Current weather 

In [2]:
# Groq Tool Calling
# Example using duckduckgo_search
from duckduckgo_search import DDGS

# Groq Tool Calling
import os
import json
from groq import Groq

from dotenv import load_dotenv

# Initialize the Groq client
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
client = Groq(api_key=groq_api_key)

# Specify the model to be used 
MODEL = 'llama-3.3-70b-versatile'

def search_internet(question):
    """
    Searches the internet for the given question.
    """
    try:
        # Perform the search using DDGS
        results = DDGS().text(question, max_results=5)
        return json.dumps(results)
    except Exception as e:
        print(f'Error occurred during search: {e}')
        return json.dumps({"error": "Failed to perform search"})

# imports calculate function from step 1
def run_conversation(user_prompt):
    # Initialize the conversation with system and user messages
    messages=[
        {
            "role": "system",
            "content": "You are a tool use assistant. \
                Always use the search_internet function to answer questions or retrieve information from the web. \
                Do not try to answer without using this tool. \
                Answer in Portuguese"

        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    # Define the available tools (i.e. functions) for our model to use
    tools = [
        {
            "type": "function",
            "function": {
                "name": "search_internet",
                "description": "Get the latest information from the internet by searching",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "The question you want to search on the internet",
                        }
                    },
                    "required": ["question"],
                },
            },
        }
    ]
    # Make the initial API call to Groq
    response = client.chat.completions.create(
        model=MODEL, # LLM to use
        messages=messages, # Conversation history
        stream=False,
        tools=tools, # Available tools (i.e. functions) for our LLM to use
        tool_choice="auto", # Let our LLM decide when to use tools
        max_tokens=4096 # Maximum number of tokens to allow in our response
    )
    # Extract the response and any tool call responses
    response_message = response.choices[0].message
    print('## Response Message:', response_message)
    tool_calls = response_message.tool_calls
    if tool_calls:
        # Define the available tools that can be called by the LLM
        available_functions = {
            "search_internet": search_internet,
        }
        # Add the LLM's response to the conversation
        messages.append(response_message)

        # Process each tool call
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            # Call the tool and get the response
            function_response = function_to_call(
                question=function_args.get("question")
            )
            # Add the tool response to the conversation
            messages.append(
                {
                    "tool_call_id": tool_call.id, 
                    "role": "tool", # Indicates this message is from tool use
                    "name": function_name,
                    "content": function_response,
                }
            )
        # Make a second API call with the updated conversation
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        # Return the final response
        return second_response.choices[0].message.content
    
# Example usage
user_prompt = "What is the weather prevision in Curitiba, Paraná in Brazil?"
print(run_conversation(user_prompt))

## Response Message: ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9ggc', function=Function(arguments='{"question": "Previsão do tempo em Curitiba, Paraná, Brasil"}', name='search_internet'), type='function')])
A previsão do tempo em Curitiba, Paraná, Brasil é de temperaturas entre 18°C e 25°C, com possibilidade de chuva. A temperatura máxima é de 25°C. É recomendável conferir sites de meteorologia como Climatempo ou AccuWeather para obter informações atualizadas sobre a previsão do tempo.


## **Parallel Tool Use**

Workflow where multiple tools can be called simultaneously, enabling more efficient and effective responses.

> _Note: Parallel tool use is natively enabled for all Llama 3 and Llama 3.1 models!_

In [ ]:
# Parallel Tool Example - tool for getting temperature and a tool for getting weather condition
# Define weather tools
def get_temperature(location: str):
    # This is a mock tool/function. In a real scenario, you would call a weather API.
    temperatures = {"New York": 22, "London": 18, "Tokyo": 26, "Sydney": 20}
    return temperatures.get(location, "Temperature data not available")

def get_weather_condition(location: str):
    # This is a mock tool/function. In a real scenario, you would call a weather API.
    conditions = {"New York": "Sunny", "London": "Rainy", "Tokyo": "Cloudy", "Sydney": "Clear"}
    return conditions.get(location, "Weather condition data not available")

# Define system messages and tools
messages = [
    {"role": "system", "content": "You are a helpful weather assistant."},
    {"role": "user", "content": "What's the weather like in New York and London?"},
]

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_temperature",
            "description": "Get the temperature for a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The name of the city",
                    }
                },
                "required": ["location"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather_condition",
            "description": "Get the weather condition for a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The name of the city",
                    }
                },
                "required": ["location"],
            },
        },
    }
]

# Make the initial request
response = client.chat.completions.create(
    model=MODEL, messages=messages, tools=tools, tool_choice="auto", max_tokens=4096
)

response_message = response.choices[0].message
tool_calls = response_message.tool_calls

# Process tool calls
messages.append(response_message)

available_functions = {
    "get_temperature": get_temperature,
    "get_weather_condition": get_weather_condition,
}

for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(**function_args)

    messages.append(
        {
            "role": "tool",
            "content": str(function_response),
            "tool_call_id": tool_call.id,
        }
    )

# Make the final request with tool call results
final_response = client.chat.completions.create(
    model=MODEL, messages=messages, tools=tools, tool_choice="auto", max_tokens=4096
)

print(final_response.choices[0].message.content)

The weather in New York is sunny and the weather in London is rainy.


#### **Tool Use with Structured Outputs**

Benefits of Using Structured Outputs

- Type Safety: Pydantic models ensure that output adheres to the expected structure, reducing the risk of errors.
- Automatic Validation: Instructor automatically validates the model's output against the defined schema.

In [3]:
import instructor
from pydantic import BaseModel, Field

# Define the tool schema
tool_schema = {
    "name": "get_weather_info",
    "description": "Get the weather information for any location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The location for which we want to get the weather information (e.g., New York)"
            }
        },
        "required": ["location"]
    }
}

# Define the Pydantic model for the tool call
class ToolCall(BaseModel):
    input_text: str = Field(description="The user's input text")
    tool_name: str = Field(description="The name of the tool to call")
    tool_parameters: str = Field(description="JSON string of tool parameters")

class ResponseModel(BaseModel):
    tool_calls: list[ToolCall]

# Patch Groq() with instructor
client = instructor.from_groq(Groq(), mode=instructor.Mode.JSON)

def run_conversation(user_prompt):
    # Prepare the messages
    messages = [
        {
            "role": "system",
            "content": f"You are an assistant that can use tools. You have access to the following tool: {tool_schema}"
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]

    # Make the Groq API call
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        response_model=ResponseModel,
        messages=messages,
        temperature=0.7,
        max_tokens=1000,
    )

    return response.tool_calls

# Example usage
user_prompt = "What's the weather like in San Francisco?"
tool_calls = run_conversation(user_prompt)

for call in tool_calls:
    print(f"Input: {call.input_text}")
    print(f"Tool: {call.tool_name}")
    print(f"Parameters: {call.tool_parameters}")
    print()

Input: What's the weather like in San Francisco?
Tool: get_weather_info
Parameters: {"location": "San Francisco"}



The purpose of the code is not to immediately answer the user's prompt but rather to identify that a tool (in this case, get_weather_info) needs to be invoked to retrieve the required information.